In [23]:
with open("AoC_7_input.txt") as f:
    content = f.read()
    szamok = content.split(',')
    szamok = [int(x) for x in szamok]
#print(szamok)

In [24]:
#szamok = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]

In [25]:
import itertools
permutaciok_tuple = list(itertools.permutations([5, 6, 7, 8, 9]))
permutaciok = []
for tuples in permutaciok_tuple:
        permutaciok.append(list(tuples))      
print(len(permutaciok))

120


In [26]:
from copy import deepcopy

In [27]:
def intcode_computer(szamok, opcode, values):
    parameterek = {1: 3, 2: 3, 3: 1, 4: 1, 5: 2, 6: 2, 7: 3, 8: 3, 99: 0, 9: 1}
    relative_base = 0
    output = []
    while opcode < len(szamok):
        ugras = True
        eltarolt_parameterek = []
        parancs = int(str(szamok[opcode])[-2::])
        i = 1
        for parameter in str(szamok[opcode])[-3::-1]:
            if int(parameter) == 1:
                eltarolt_parameterek.append(opcode + i)
            elif int(parameter) == 0:
                eltarolt_parameterek.append(szamok[opcode + i])
            elif int(parameter) == 2:
                eltarolt_parameterek.append(szamok[opcode + i] + relative_base)
            i += 1

        for _ in range(parameterek[int(str(szamok[opcode])[-2::])] - len(eltarolt_parameterek)):
            eltarolt_parameterek.append(szamok[opcode + i])
            i += 1

        if parancs != 99 and len(szamok) <= max(eltarolt_parameterek):
            nullas = [0]
            szamok = szamok + nullas * (max(eltarolt_parameterek)-len(szamok) + 1)

        if parancs == 1:
            szamok[eltarolt_parameterek[2]] = szamok[eltarolt_parameterek[0]] + szamok[eltarolt_parameterek[1]]

        elif parancs == 2:
            szamok[eltarolt_parameterek[2]] = szamok[eltarolt_parameterek[0]] * szamok[eltarolt_parameterek[1]]

        elif parancs == 3:
            if values != []:
                szamok[eltarolt_parameterek[0]] = values[0]
                values.pop(0)
            else:
                return [szamok, opcode, output, True]
                
        elif parancs == 4:
            output.append(szamok[eltarolt_parameterek[0]])

        elif parancs == 5:
            if szamok[eltarolt_parameterek[0]] != 0:
                opcode = szamok[eltarolt_parameterek[1]]
                ugras = False

        elif parancs == 6:
            if szamok[eltarolt_parameterek[0]] == 0:
                opcode = szamok[eltarolt_parameterek[1]]
                ugras = False

        elif parancs == 7:
            if szamok[eltarolt_parameterek[0]] < szamok[eltarolt_parameterek[1]]:
                szamok[eltarolt_parameterek[2]] = 1
            else:
                szamok[eltarolt_parameterek[2]] = 0

        elif parancs == 8:
            if szamok[eltarolt_parameterek[0]] == szamok[eltarolt_parameterek[1]]:
                szamok[eltarolt_parameterek[2]] = 1
            else:
                szamok[eltarolt_parameterek[2]] = 0

        elif parancs == 9:
            relative_base += szamok[eltarolt_parameterek[0]]

        elif szamok[opcode] == 99:
            return [szamok, opcode, output, False]

        if ugras:
            opcode += parameterek[int(str(szamok[opcode])[-2::])] + 1 

In [28]:
thrusters = []
for phase_list in permutaciok:
    futas = True
    key_index = 1
    amp_dict = {1: {'szamok': deepcopy(szamok), 'opcode': 0, 'inputok': [phase_list[0], 0]}, 
            2: {'szamok': deepcopy(szamok), 'opcode': 0, 'inputok': [phase_list[1]]}, 
            3: {'szamok': deepcopy(szamok), 'opcode': 0, 'inputok': [phase_list[2]]}, 
            4: {'szamok': deepcopy(szamok), 'opcode': 0, 'inputok': [phase_list[3]]}, 
            5: {'szamok': deepcopy(szamok), 'opcode': 0, 'inputok': [phase_list[4]]}}
    while futas:
        return_ertekek = intcode_computer(amp_dict[key_index]['szamok'], amp_dict[key_index]['opcode'], amp_dict[key_index]['inputok'])
        amp_dict[key_index] = {'szamok': return_ertekek[0], 'opcode': return_ertekek[1], 'inputok': []}
        futas = return_ertekek[3]
        if futas == False and key_index != 5:
            futas = True
        key_index += 1
        if key_index > 5:
            key_index = 1
        amp_dict[key_index]['inputok'] += return_ertekek[2]
    thrusters.append(return_ertekek[2][0])

In [29]:
max(thrusters)

36497698

1. az 5 amplifiernek külön kell tárolni az információkat (intcodjait, indexét a amplifiernek, milyen inputjai vannak)
2. intcode-on belül át kell írni az outputos és inputos és 99-es parancsot
3. while